In [2]:
from contgan_images_simple.contgan import ContGAN
from contgan_images_simple.utils import load_weights

device = "cuda:7"
model = ContGAN(image_shape=(3, 32, 32))
load_weights(model, "model.pt")

model.to(device)
model.eval();

In [5]:
from contgan_images.utils import jvp

import torch.nn as nn
import torch as th

from functools import partial

class ODEfunc(nn.Module):
    def __init__(self, model):
        super().__init__()

        self.model = model

    def forward(self, t, x):
        t = t.unsqueeze(0)
        return self.model.direction(x_t=x, t=t)
    
ode_func = ODEfunc(model)

In [6]:
from torchdiffeq import odeint_adjoint as odeint
import numpy as np

out = odeint(ode_func, th.randn((64, 3, 32, 32), device=device), th.tensor(np.linspace(1., 0., 2), device=device))
out = (out[1].clip(-1., 1.) + 1.) / 2.

In [7]:
from torchvision.utils import save_image

save_image(out, "out/contgan_images_simple_ode_sample.png")

---